In [1]:
from AOC_utils import get_day
import numpy as np

input_data = get_day(day=8, year=2025)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 8 input already downloaded
1000


['66808,15875,57273',
 '56891,2658,63198',
 '67170,13852,22738',
 '34042,76706,39911',
 '48789,35433,67250',
 '16982,1436,35054',
 '30664,71747,26236']

In [2]:
example = '''
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
'''.split('\n')[1:-1]

In [3]:
def solve_day(data, n_connections=-1):
    part1 = 0; part2 = 0

    data = np.array([[int(y) for y in x.split(',')] for x in data], dtype=np.float32)

    distances = np.linalg.norm(data[:, np.newaxis, :] - data[np.newaxis, :, :], axis=-1)
    triangle_indices = np.triu_indices(len(data), k=1)
    distances = distances[triangle_indices]
    distances_argsort = np.argsort(distances)

    indicies_smallest_to_largest = (triangle_indices[0][distances_argsort], triangle_indices[1][distances_argsort])

    network = {i: set() for i in range(len(data))}
    for i in range(len(data)):
        network[i].add(i)

    for step, (i, j) in enumerate(zip(*indicies_smallest_to_largest)):
        already_connected = False
        smallest_id = min(i, j)
        smallest_has_been_found = False
        for k, connections in network.items():
            if i in connections or j in connections:
                if not smallest_has_been_found: smallest_id = min(smallest_id, k)
                smallest_has_been_found = True
                network[smallest_id] = network[smallest_id].union(network[i])
                network[smallest_id] = network[smallest_id].union(network[j])
                network[smallest_id] = network[smallest_id].union(network[k])

                network[smallest_id].add(i)
                network[smallest_id].add(j)
                network[smallest_id].add(k)

                if i != smallest_id:
                    network[i] = set()
                if j != smallest_id:
                    network[j] = set()
                if k != smallest_id:
                    network[k] = set()
                already_connected = True
                
        if not already_connected:
            smaller_id = min(i, j)
            larger_id = max(i, j)
            
            network[smaller_id] = network[smaller_id].union(network[larger_id])
            network[smaller_id].add(larger_id)
            network[larger_id] = set()

        if len(network[smallest_id]) == len(data):
            part2 = int(data[i][0]) * int(data[j][0])
            break

        if step == n_connections - 1:
            sorted_network = dict(sorted(network.items(), key=lambda item: len(item[1]), reverse=True))
            part1 = 1
            x = 0
            for connections in sorted_network.values():
                if x == 3:
                    break
                part1 *= len(connections)
                x += 1

    print("part 1:", part1)
    print("part 2:", part2)

solve_day(example, n_connections=10)

part 1: 40
part 2: 25272


In [4]:
solve_day(input_data, n_connections=1000)

part 1: 181584
part 2: 8465902405
